# Environment

In [1]:
import sys
from os import path
sys.path.append(path.dirname(path.dirname(path.abspath("Test.ipynb"))))
import time
import logging
from Controllers_Definition import testcontroller1, testcontroller2, testcontroller3, testcontroller4
from datetime import datetime
from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack
import random
import threading
import multiprocessing as mp

In [2]:
logger = logging.getLogger(__name__)
'''
import matplotlib.pyplot as plt
%matplotlib inline
'''
logger.setLevel(logging.DEBUG)

# Triggering Class

In [3]:
ts = {} 
ts['main'] = 0.5 # seconds
ts['print'] = 1 # seconds

trigger_test = triggering(ts)
now_init = time.time()
now = now_init
while now < now_init+2:
    now = time.time()
    if now >= trigger_test.trigger['main']:
        print ('Main triggered', now)
        trigger_test.refresh_trigger('main', now)
    if now >= trigger_test.trigger['print']:
        print ('Print triggered', now)
        trigger_test.refresh_trigger('print', now)

Main triggered 1614074754.0
Print triggered 1614074754.0
Main triggered 1614074754.5
Main triggered 1614074755.0
Print triggered 1614074755.0


# Controller Base Class (eFMU)

In [4]:
# Test controller
testcontroller = testcontroller1() #this controller multiplies the inputs
# Get all variables
variables = testcontroller.get_model_variables()
# Makeup some inputs
inputs = {}
for var in variables:
    inputs[var] = random.randint(1,50)
# Query controller
print ('Log-message', testcontroller.do_step(inputs=inputs))
print ('Input', testcontroller.input)
print ('Output', testcontroller.output)
print('Output', testcontroller.get_var('output'))

Log-message testcontroller1 did a computation!
Input {'a': 28, 'b': 43}
Output {'c': 1204}
Output {'c': 1204}


# Controller Stack Class (single-thread/multi-thread)

In [5]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'

## Single Thread

In [6]:
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=False, timeout=2)

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
for i in range(6):
    ctrl_stack.query_control(time.time())
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:01.196395008,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:03.473878016,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:05.748466944,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:08.022716928,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:10.299760128,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:12.571396096,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:01.196395008,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:03.473878016,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:05.748466944,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:08.022716928,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:10.299760128,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:12.571396096,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:01.196395008,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:03.473878016,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:05.748466944,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:08.022716928,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:10.299760128,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:12.571396096,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:01.196395008,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:03.473878016,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:05.748466944,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:08.022716928,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:10.299760128,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:12.571396096,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:01.196395008,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:03.473878016,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:05.748466944,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:08.022716928,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:10.299760128,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:12.571396096,30.0,4.0,120.0,testcontroller1 did a computation!


13.81995701789856


## Multi Thread

In [7]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2)


# Call query_control 6 times. We should expect there are 6 records(excluding NaN) for each task.
# In multi thread mod, each call of query_control will trigger a computation for one controller within each task. 
# We assign tasks based on input dependency. Since the inputs of mpc1 and control1 depend on the out put of
# forecast1, they are in the same task as forecast1.
t = time.time()
while time.time() - t < 10:
    ctrl_stack.query_control(time.time())
for df in ctrl_stack.log_to_df().values():
    display(df)

,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:20.837776896,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:22.096636928,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:23.350816000,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:24.605831936,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:25.882283008,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:27.135436032,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:28.391837952,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:29.647054080,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:20.837776896,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:22.096636928,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:23.350816000,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:24.605831936,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:25.882283008,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:27.135436032,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:28.391837952,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:29.647054080,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:20.837776896,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:22.096636928,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:23.350816000,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:24.605831936,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:25.882283008,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:27.135436032,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:28.391837952,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:29.647054080,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:20.837776896,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:22.096636928,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:23.350816000,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:24.605831936,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:25.882283008,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:27.135436032,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:28.391837952,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:29.647054080,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:20.837776896,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:22.096636928,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:23.350816000,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:24.605831936,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:25.882283008,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:27.135436032,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:28.391837952,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:29.647054080,30.0,4.0,120.0,testcontroller1 did a computation!


In [8]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2)

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
threads = []
while time.time() - t < 10:
    threads.append(threading.Thread(target=controller_stack.query_control, args=(ctrl_stack, time.time())))
    threads[-1].start()
print(time.time() - t)
for thread in threads:
    thread.join()

10.00088119506836


In [9]:
for df in ctrl_stack.log_to_df().values():
    display(df)

,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:36.987738112,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:37.990272000,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:38.992146944,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:39.993326080,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:40.994199040,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:41.995217920,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:42.999038976,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:44.000353024,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-23 02:06:45.001541120,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:36.987738112,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:37.990272000,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:38.992146944,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:39.993326080,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:40.994199040,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:41.995217920,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:42.999038976,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:44.000353024,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-23 02:06:45.001541120,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:36.987738112,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:37.990272000,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:38.992146944,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:39.993326080,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:40.994199040,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:41.995217920,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:42.999038976,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:44.000353024,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-23 02:06:45.001541120,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:36.987738112,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:37.990272000,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:38.992146944,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:39.998059008,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:40.999671040,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:42.000008960,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:43.005699072,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:44.014984960,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-23 02:06:45.015024896,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-23 02:05:53.358153984,NaN,NaN,NaN,Initialize
2021-02-23 02:06:36.987738112,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:37.990272000,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:38.992146944,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:39.998059008,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:41.003203072,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:42.009314048,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:43.009444096,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:44.014984960,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-23 02:06:45.016584960,30.0,4.0,120.0,testcontroller1 did a computation!
